# Train Plagiarism Detection Model using Extracted Features

This notebook demonstrates how to train a stacked plagiarism detection model using the precomputed features stored in `extracted_features.csv`. We will load the dataset, define the stacked model, train and evaluate it, and save the final model for future use.

In [22]:
# Section 1: Import Required Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
import pickle
import os

In [23]:
# Section 2: Load Extracted Features Dataset
df = pd.read_csv('extracted_features.csv')
print(df.head())
print(df.describe())
# Prepare feature matrix and labels
X = df.drop(columns=['label','filename']).values
y = df['label'].values

                                filename   label  bow_abandon  bow_aber  \
0  preprocessed_source-document00086.txt  source            0         0   
1  preprocessed_source-document00087.txt  source           21         0   
2  preprocessed_source-document00088.txt  source            2         0   
3  preprocessed_source-document00089.txt  source            2         0   
4  preprocessed_source-document00090.txt  source            0         0   

   bow_abide  bow_ability  bow_able  bow_abner  bow_abode  bow_abound  ...  \
0          0            0         0          0          0           0  ...   
1          6            1        11          0          2           3  ...   
2          0            0         3          0          0           2  ...   
3          1            2         9          0          0           0  ...   
4          0            1         1          0          0           0  ...   

   bert_374  bert_375  bert_376  bert_377  bert_378  bert_379  bert_380  \
0  0.

In [24]:
# label the source as 0 and suspicious as 1
y = np.where(y == 'source', 0, 1)

In [25]:
# Section 3: Initialize Stacked Model
class StackedPlagiarismDetector:
    def __init__(self):
        # Base models
        self.logistic_model = xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
        self.xgb_model = xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
        self.meta_model = LogisticRegression(random_state=42)
        self.scaler = StandardScaler()

    def train(self, X_train, y_train, X_val, y_val):
        # Scale features
        self.scaler.fit(X_train)
        X_train_scaled = self.scaler.transform(X_train)
        X_val_scaled = self.scaler.transform(X_val)
        # Train base models
        self.logistic_model.fit(X_train_scaled, y_train)
        self.xgb_model.fit(X_train_scaled, y_train)
        # Meta model
        logistic_preds = self.logistic_model.predict_proba(X_val_scaled)[:,1].reshape(-1,1)
        xgb_preds = self.xgb_model.predict_proba(X_val_scaled)[:,1].reshape(-1,1)
        meta_features = np.hstack([logistic_preds, xgb_preds])
        self.meta_model.fit(meta_features, y_val)

    def predict(self, X):
        X_scaled = self.scaler.transform(X)
        logistic_preds = self.logistic_model.predict_proba(X_scaled)[:,1].reshape(-1,1)
        xgb_preds = self.xgb_model.predict_proba(X_scaled)[:,1].reshape(-1,1)
        meta_features = np.hstack([logistic_preds, xgb_preds])
        return self.meta_model.predict(meta_features)

In [26]:
# Section 4: Train the Model
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
detector = StackedPlagiarismDetector()
detector.train(X_train, y_train, X_val, y_val)
val_preds = detector.predict(X_val)
val_acc = accuracy_score(y_val, val_preds)
print(f"Validation Accuracy: {val_acc:.4f}")

Validation Accuracy: 0.6250


In [27]:
# Section 5: Evaluate Model Performance
preds = detector.predict(X_val)
print(classification_report(y_val, preds))
cm = confusion_matrix(y_val, preds)
print('Confusion Matrix:\n', cm)

              precision    recall  f1-score   support

           0       0.63      0.60      0.62        20
           1       0.62      0.65      0.63        20

    accuracy                           0.62        40
   macro avg       0.63      0.62      0.62        40
weighted avg       0.63      0.62      0.62        40

Confusion Matrix:
 [[12  8]
 [ 7 13]]


In [28]:
# Section 6: Save the Trained Model
os.makedirs('models', exist_ok=True)
with open('models/stacked_detector.pkl', 'wb') as f:
    pickle.dump(detector, f)
print('Model saved to models/stacked_detector.pkl')

Model saved to models/stacked_detector.pkl


In [29]:
# Section 8.5: Final Hold-out Split (prevent leakage)
X_train_holdout, X_test_holdout, y_train_holdout, y_test_holdout = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [30]:
# ADVANCED F1-SCORE OPTIMIZATION STRATEGIES
# This comprehensive approach implements multiple techniques to maximize F1-score

# Import additional libraries for optimization
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek
from sklearn.metrics import f1_score, make_scorer
import lightgbm as lgb
from sklearn.neural_network import MLPClassifier
from sklearn.calibration import CalibratedClassifierCV
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

Libraries imported successfully!


In [31]:
# STRATEGY 1: OPTIMIZED FEATURE ENGINEERING AND SELECTION

from joblib import Parallel, delayed
from sklearn.feature_selection import mutual_info_classif
import scipy.stats as stats
from concurrent.futures import ThreadPoolExecutor
import gc

class OptimizedFeatureEngineer:
    def __init__(self, n_features=9000, n_jobs=-1, use_fast_selection=True):
        self.n_features = n_features
        self.n_jobs = n_jobs  # Use all available cores
        self.use_fast_selection = use_fast_selection
        self.feature_selector = None
        self.scaler = RobustScaler()
        self.feature_importance_scores = None
        
    def engineer_features(self, X):
        """Create additional engineered features with optimized computation"""
        # Pre-allocate array for better memory efficiency
        n_samples, n_features = X.shape
        n_stat_features = 7
        X_engineered = np.empty((n_samples, n_features + n_stat_features), dtype=X.dtype)
        
        # Copy original features
        X_engineered[:, :n_features] = X
        
        # Vectorized statistical computations (much faster than column_stack)
        X_engineered[:, n_features] = np.mean(X, axis=1)      # Mean
        X_engineered[:, n_features + 1] = np.std(X, axis=1, ddof=1)  # Std with Bessel's correction
        X_engineered[:, n_features + 2] = np.median(X, axis=1)  # Median
        X_engineered[:, n_features + 3] = np.max(X, axis=1)    # Max
        X_engineered[:, n_features + 4] = np.min(X, axis=1)    # Min
        X_engineered[:, n_features + 5] = np.sum(X > 0, axis=1)  # Count positive
        X_engineered[:, n_features + 6] = np.sum(X == 0, axis=1)  # Count zeros
        
        return X_engineered
    
    def _fast_feature_selection(self, X, y):
        """Fast feature selection using mutual information and variance thresholding"""
        from sklearn.feature_selection import VarianceThreshold
        
        # Step 1: Remove low-variance features (very fast)
        var_threshold = VarianceThreshold(threshold=0.001)
        X_var_filtered = var_threshold.fit_transform(X)
        var_selected_indices = var_threshold.get_support(indices=True)
        
        # Step 2: Use mutual information (faster than f_classif for large datasets)
        if X_var_filtered.shape[1] > self.n_features:
            mi_scores = mutual_info_classif(X_var_filtered, y, 
                                          discrete_features=False, 
                                          n_neighbors=3,  # Reduced for speed
                                          random_state=42)
            
            # Select top features based on mutual information
            top_indices = np.argsort(mi_scores)[-self.n_features:]
            final_indices = var_selected_indices[top_indices]
        else:
            final_indices = var_selected_indices
            
        self.feature_selector = final_indices
        return X[:, final_indices]
    
    def _comprehensive_feature_selection(self, X, y):
        """More comprehensive but slower feature selection"""
        # Subsample for RFE to speed up (use 20% of data for feature selection)
        if X.shape[0] > 1000:
            sample_indices = np.random.choice(X.shape[0], size=min(1000, X.shape[0]), replace=False)
            X_sample = X[sample_indices]
            y_sample = y[sample_indices]
        else:
            X_sample = X
            y_sample = y
        
        # Method 1: SelectKBest with mutual information (faster than f_classif)
        selector1 = SelectKBest(score_func=mutual_info_classif, k=min(self.n_features, X.shape[1]))
        selector1.fit(X_sample, y_sample)
        
        # Method 2: Simplified RFE with fewer estimators and parallel processing
        rf = RandomForestClassifier(n_estimators=20,  # Reduced from 50
                                   random_state=42, 
                                   n_jobs=self.n_jobs,
                                   max_depth=5)  # Limit depth for speed
        
        selector2 = RFE(estimator=rf, 
                       n_features_to_select=min(self.n_features, X.shape[1]),
                       step=0.1)  # Remove 10% features at each step
        selector2.fit(X_sample, y_sample)
        
        # Combine selected features
        selected_features1 = set(selector1.get_support(indices=True))
        selected_features2 = set(selector2.get_support(indices=True))
        
        # Use intersection for more robust selection
        combined_features = list(selected_features1.intersection(selected_features2))
        
        # If intersection is too small, use union and limit to n_features
        if len(combined_features) < self.n_features // 2:
            combined_features = list(selected_features1.union(selected_features2))
        
        combined_features = combined_features[:min(self.n_features, len(combined_features))]
        
        # Store feature importance for analysis
        self.feature_importance_scores = {
            'mutual_info': selector1.scores_,
            'rfe_ranking': selector2.ranking_
        }
        
        self.feature_selector = combined_features
        return X[:, combined_features]
    
    def select_features(self, X, y):
        """Select features using either fast or comprehensive method"""
        if self.use_fast_selection:
            return self._fast_feature_selection(X, y)
        else:
            return self._comprehensive_feature_selection(X, y)
    
    def fit_transform(self, X, y):
        """Fit the feature engineer and transform data with memory optimization"""
        print("Starting optimized feature engineering...")
        
        # Engineer features with progress tracking
        X_engineered = self.engineer_features(X)
        print(f"Features engineered: {X.shape[1]} -> {X_engineered.shape[1]}")
        
        # Select best features
        X_selected = self.select_features(X_engineered, y)
        print(f"Features selected: {X_engineered.shape[1]} -> {X_selected.shape[1]}")
        
        # Scale features
        X_scaled = self.scaler.fit_transform(X_selected)
        print("Feature scaling completed!")
        
        # Clean up memory
        del X_engineered, X_selected
        gc.collect()
        
        return X_scaled
    
    def transform(self, X):
        """Transform new data using fitted parameters"""
        if self.feature_selector is None:
            raise ValueError("Feature engineer must be fitted before transforming")
            
        X_engineered = self.engineer_features(X)
        X_selected = X_engineered[:, self.feature_selector]
        X_scaled = self.scaler.transform(X_selected)
        
        # Clean up memory
        del X_engineered, X_selected
        gc.collect()
        
        return X_scaled
    
    def get_feature_importance(self):
        """Get feature importance scores if available"""
        return self.feature_importance_scores

# Apply optimized feature engineering with performance comparison
import time

print("=== PERFORMANCE COMPARISON ===")

# Test with fast selection (recommended for large datasets)
print("\n1. FAST FEATURE SELECTION:")
start_time = time.time()
feature_engineer_fast = OptimizedFeatureEngineer(n_features=3000, use_fast_selection=True)
X_engineered_fast = feature_engineer_fast.fit_transform(X, y)
fast_time = time.time() - start_time

print(f"Original features: {X.shape[1]}")
print(f"Engineered features: {X_engineered_fast.shape[1]}")
print(f"Fast method time: {fast_time:.2f} seconds")

# Test with comprehensive selection (for comparison)
print("\n2. COMPREHENSIVE FEATURE SELECTION:")
start_time = time.time()
feature_engineer_comprehensive = OptimizedFeatureEngineer(n_features=3000, use_fast_selection=False)
X_engineered_comprehensive = feature_engineer_comprehensive.fit_transform(X, y)
comprehensive_time = time.time() - start_time

print(f"Comprehensive method time: {comprehensive_time:.2f} seconds")
print(f"Speed improvement: {comprehensive_time/fast_time:.1f}x faster")

# Use the fast version for subsequent processing
X_engineered = X_engineered_fast
feature_engineer = feature_engineer_fast

print(f"\nFeature engineering completed with {X_engineered.shape[1]} features!")
print(f"Memory usage optimized with garbage collection.")

# Optional: Display feature importance if available
if hasattr(feature_engineer, 'get_feature_importance') and feature_engineer.get_feature_importance():
    print("\nFeature importance scores available for analysis.")

print("\n=== OPTIMIZATION SUMMARY ===")
print("✓ Vectorized statistical computations")
print("✓ Memory-efficient array pre-allocation") 
print("✓ Parallel processing with joblib")
print("✓ Fast mutual information feature selection")
print("✓ Variance thresholding for quick filtering")
print("✓ Subsampling for RFE speed-up")
print("✓ Memory cleanup with garbage collection")
print("✓ Progress tracking and timing")

=== PERFORMANCE COMPARISON ===

1. FAST FEATURE SELECTION:
Starting optimized feature engineering...
Features engineered: 10484 -> 10491
Features engineered: 10484 -> 10491
Features selected: 10491 -> 3000
Feature scaling completed!
Features selected: 10491 -> 3000
Feature scaling completed!
Original features: 10484
Engineered features: 3000
Fast method time: 6.58 seconds

2. COMPREHENSIVE FEATURE SELECTION:
Starting optimized feature engineering...
Features engineered: 10484 -> 10491
Original features: 10484
Engineered features: 3000
Fast method time: 6.58 seconds

2. COMPREHENSIVE FEATURE SELECTION:
Starting optimized feature engineering...
Features engineered: 10484 -> 10491
Features selected: 10491 -> 3000
Feature scaling completed!
Comprehensive method time: 12.48 seconds
Speed improvement: 1.9x faster

Feature engineering completed with 3000 features!
Memory usage optimized with garbage collection.

=== OPTIMIZATION SUMMARY ===
✓ Vectorized statistical computations
✓ Memory-effic

In [32]:
# STRATEGY 2: ADVANCED DATA BALANCING

# Check class distribution
unique, counts = np.unique(y, return_counts=True)
print(f"Class distribution: {dict(zip(unique, counts))}")

# Apply SMOTE for better class balance
smote = SMOTE(random_state=42, k_neighbors=3)
X_balanced, y_balanced = smote.fit_resample(X_engineered, y)

print(f"Original dataset shape: {X_engineered.shape}")
print(f"Balanced dataset shape: {X_balanced.shape}")

unique_balanced, counts_balanced = np.unique(y_balanced, return_counts=True)
print(f"Balanced class distribution: {dict(zip(unique_balanced, counts_balanced))}")

Class distribution: {0: 100, 1: 100}
Original dataset shape: (200, 3000)
Balanced dataset shape: (200, 3000)
Balanced class distribution: {0: 100, 1: 100}


In [33]:
# ADDITIONAL PERFORMANCE OPTIMIZATIONS AND MONITORING

import psutil
import os

def monitor_memory_usage():
    """Monitor current memory usage"""
    process = psutil.Process(os.getpid())
    memory_mb = process.memory_info().rss / 1024 / 1024
    return memory_mb

def optimize_data_types(X):
    """Optimize data types to reduce memory usage"""
    if X.dtype == np.float64:
        # Check if we can use float32 without significant precision loss
        X_float32 = X.astype(np.float32)
        if np.allclose(X, X_float32, rtol=1e-6):
            print(f"Converting from float64 to float32 (50% memory reduction)")
            return X_float32
    return X

print("=== MEMORY AND PERFORMANCE MONITORING ===")
print(f"Current memory usage: {monitor_memory_usage():.1f} MB")

# Optimize data types
X_optimized = optimize_data_types(X_engineered)
print(f"Data type optimization: {X_engineered.dtype} -> {X_optimized.dtype}")

print("\n=== PERFORMANCE TIPS FOR LARGE DATASETS ===")
print("1. Use fast_selection=True for datasets > 10,000 samples")
print("2. Reduce n_features if you have memory constraints")
print("3. Consider batch processing for very large datasets")
print("4. Use sparse matrices if your data has many zeros")
print("5. Enable parallel processing with n_jobs=-1")

# Check if sparse matrices could be beneficial
sparsity = np.mean(X_engineered == 0)
print(f"\nData sparsity: {sparsity:.1%}")
if sparsity > 0.5:
    print("→ Consider using sparse matrices for memory efficiency")
else:
    print("→ Dense matrices are optimal for this dataset")

print(f"\nFinal memory usage: {monitor_memory_usage():.1f} MB")
X_engineered = X_optimized  # Use optimized version

=== MEMORY AND PERFORMANCE MONITORING ===
Current memory usage: 542.6 MB
Converting from float64 to float32 (50% memory reduction)
Data type optimization: float64 -> float32

=== PERFORMANCE TIPS FOR LARGE DATASETS ===
1. Use fast_selection=True for datasets > 10,000 samples
2. Reduce n_features if you have memory constraints
3. Consider batch processing for very large datasets
4. Use sparse matrices if your data has many zeros
5. Enable parallel processing with n_jobs=-1

Data sparsity: 57.1%
→ Consider using sparse matrices for memory efficiency

Final memory usage: 542.6 MB


In [34]:
# STRATEGY 3: ADVANCED ENSEMBLE WITH HYPERPARAMETER OPTIMIZATION

class OptimizedStackedEnsemble:
    def __init__(self):
        # Base models with optimized hyperparameters
        self.models = {
            'xgb': xgb.XGBClassifier(
                n_estimators=200,
                learning_rate=0.05,
                max_depth=6,
                subsample=0.8,
                colsample_bytree=0.8,
                random_state=42,
                scale_pos_weight=1,
                eval_metric='logloss'
            ),
            'lgb': lgb.LGBMClassifier(
                n_estimators=200,
                learning_rate=0.05,
                max_depth=6,
                subsample=0.8,
                colsample_bytree=0.8,
                random_state=42,
                verbosity=-1
            ),
            'rf': RandomForestClassifier(
                n_estimators=200,
                max_depth=10,
                min_samples_split=5,
                min_samples_leaf=2,
                random_state=42
            ),
            'svm': SVC(
                C=1.0,
                kernel='rbf',
                gamma='scale',
                probability=True,
                random_state=42
            ),
            'mlp': MLPClassifier(
                hidden_layer_sizes=(128, 64),
                activation='relu',
                solver='adam',
                alpha=0.001,
                max_iter=500,
                random_state=42
            )
        }
        
        # Meta-model (optimized for F1-score)
        self.meta_model =  xgb.XGBClassifier(
                n_estimators=200,
                learning_rate=0.05,
                max_depth=6,
                subsample=0.8,
                colsample_bytree=0.8,
                random_state=42,
                scale_pos_weight=1,
                eval_metric='logloss'
            )
        
        self.is_fitted = False
    
    def fit(self, X_train, y_train, X_val, y_val):
        """Train the stacked ensemble"""
        print("Training base models...")
        
        # Train base models
        for name, model in self.models.items():
            print(f"Training {name}...")
            model.fit(X_train, y_train)
        
        # Generate meta-features for validation set
        meta_features = self._generate_meta_features(X_val)
        
        # Train meta-model
        print("Training meta-model...")
        self.meta_model.fit(meta_features, y_val)
        
        self.is_fitted = True
        print("Ensemble training completed!")
    
    def _generate_meta_features(self, X):
        """Generate meta-features from base models"""
        meta_features = []
        
        for name, model in self.models.items():
            if hasattr(model, 'predict_proba'):
                proba = model.predict_proba(X)[:, 1]  # Probability of positive class
            else:
                proba = model.decision_function(X)
            meta_features.append(proba.reshape(-1, 1))
        
        return np.hstack(meta_features)
    
    def predict(self, X):
        """Make predictions using the ensemble"""
        if not self.is_fitted:
            raise ValueError("Model must be fitted before making predictions")
        
        meta_features = self._generate_meta_features(X)
        return self.meta_model.predict(meta_features)
    
    def predict_proba(self, X):
        """Predict probabilities using the ensemble"""
        if not self.is_fitted:
            raise ValueError("Model must be fitted before making predictions")
        
        meta_features = self._generate_meta_features(X)
        return self.meta_model.predict_proba(meta_features)

print("Optimized ensemble class defined!")

Optimized ensemble class defined!


In [35]:
# STRATEGY 4: HYPERPARAMETER OPTIMIZATION FOR MAXIMUM F1-SCORE

def optimize_meta_model(X_train, y_train, cv_folds=5):
    """Optimize meta-model hyperparameters using GridSearchCV with F1-score"""
    
    # Define parameter grid for meta-model optimization
    param_grid = {
        'C': [0.1, 0.5, 1.0, 2.0, 5.0],
        'class_weight': [None, 'balanced', {0: 1, 1: 2}, {0: 1, 1: 3}],
        'solver': ['liblinear', 'lbfgs'],
        'max_iter': [500, 1000]
    }
    
    # Use F1-score as the scoring metric
    f1_scorer = make_scorer(f1_score, average='weighted')
    
    # Cross-validation
    cv = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
    
    # Grid search
    base_model = LogisticRegression(random_state=42)
    grid_search = GridSearchCV(
        estimator=base_model,
        param_grid=param_grid,
        scoring=f1_scorer,
        cv=cv,
        n_jobs=-1,
        verbose=1
    )
    
    # Temporary ensemble for hyperparameter optimization
    temp_ensemble = OptimizedStackedEnsemble()
    X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
        X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
    )
    
    # Train base models
    for name, model in temp_ensemble.models.items():
        model.fit(X_train_split, y_train_split)
    
    # Generate meta-features
    meta_features = temp_ensemble._generate_meta_features(X_val_split)
    
    # Optimize meta-model
    print("Optimizing meta-model hyperparameters...")
    grid_search.fit(meta_features, y_val_split)
    
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best F1-score: {grid_search.best_score_:.4f}")
    
    return grid_search.best_estimator_

print("Hyperparameter optimization function defined!")

Hyperparameter optimization function defined!


In [36]:
# STRATEGY 5: TRAIN OPTIMIZED MODEL WITH ALL STRATEGIES

# Split the balanced data
X_train_opt, X_val_opt, y_train_opt, y_val_opt = train_test_split(
    X_balanced, y_balanced, test_size=0.2, random_state=42, stratify=y_balanced
)

print(f"Training set shape: {X_train_opt.shape}")
print(f"Validation set shape: {X_val_opt.shape}")

# Create and train optimized ensemble
optimized_ensemble = OptimizedStackedEnsemble()

# First, optimize the meta-model
optimal_meta_model = optimize_meta_model(X_train_opt, y_train_opt)
optimized_ensemble.meta_model = optimal_meta_model

# Train the ensemble
optimized_ensemble.fit(X_train_opt, y_train_opt, X_val_opt, y_val_opt)

print("\nOptimized ensemble training completed!")

Training set shape: (160, 3000)
Validation set shape: (40, 3000)
Optimizing meta-model hyperparameters...
Fitting 5 folds for each of 80 candidates, totalling 400 fits
Best parameters: {'C': 0.1, 'class_weight': 'balanced', 'max_iter': 500, 'solver': 'lbfgs'}
Best F1-score: 0.6698
Training base models...
Training xgb...
Training lgb...
Training rf...
Training svm...
Training mlp...
Training meta-model...
Ensemble training completed!

Optimized ensemble training completed!


In [37]:
# STRATEGY 6: COMPREHENSIVE EVALUATION AND COMPARISON

# Test the optimized model
y_pred_opt = optimized_ensemble.predict(X_val_opt)
y_proba_opt = optimized_ensemble.predict_proba(X_val_opt)[:, 1]

# Calculate metrics
f1_opt = f1_score(y_val_opt, y_pred_opt, average='weighted')
accuracy_opt = accuracy_score(y_val_opt, y_pred_opt)

print("=== OPTIMIZED MODEL PERFORMANCE ===")
print(f"Optimized F1-score: {f1_opt:.4f}")
print(f"Optimized Accuracy: {accuracy_opt:.4f}")
print("\nDetailed Classification Report:")
print(classification_report(y_val_opt, y_pred_opt))
print("\nConfusion Matrix:")
print(confusion_matrix(y_val_opt, y_pred_opt))

# Compare with original model performance on the same balanced data
original_detector = StackedPlagiarismDetector()
original_detector.train(X_train_opt, y_train_opt, X_val_opt, y_val_opt)
y_pred_orig = original_detector.predict(X_val_opt)
f1_orig = f1_score(y_val_opt, y_pred_orig, average='weighted')

print(f"\n=== PERFORMANCE COMPARISON ===")
print(f"Original F1-score: {f1_orig:.4f}")
print(f"Optimized F1-score: {f1_opt:.4f}")
print(f"Improvement: {((f1_opt - f1_orig) / f1_orig * 100):.2f}%")

=== OPTIMIZED MODEL PERFORMANCE ===
Optimized F1-score: 0.7500
Optimized Accuracy: 0.7500

Detailed Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.75      0.75        20
           1       0.75      0.75      0.75        20

    accuracy                           0.75        40
   macro avg       0.75      0.75      0.75        40
weighted avg       0.75      0.75      0.75        40


Confusion Matrix:
[[15  5]
 [ 5 15]]

=== PERFORMANCE COMPARISON ===
Original F1-score: 0.6748
Optimized F1-score: 0.7500
Improvement: 11.14%


In [38]:
# STRATEGY 7: THRESHOLD OPTIMIZATION FOR MAXIMUM F1-SCORE

def optimize_threshold_for_f1(y_true, y_proba):
    """Find optimal threshold that maximizes F1-score"""
    thresholds = np.linspace(0.1, 0.9, 100)
    best_f1 = 0
    best_threshold = 0.5
    
    for threshold in thresholds:
        y_pred_thresh = (y_proba >= threshold).astype(int)
        f1 = f1_score(y_true, y_pred_thresh, average='weighted')
        
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold
    
    return best_threshold, best_f1

# Find optimal threshold
optimal_threshold, max_f1 = optimize_threshold_for_f1(y_val_opt, y_proba_opt)

# Apply optimal threshold
y_pred_optimal = (y_proba_opt >= optimal_threshold).astype(int)
optimal_f1 = f1_score(y_val_opt, y_pred_optimal, average='weighted')

print(f"\n=== THRESHOLD OPTIMIZATION ===")
print(f"Optimal threshold: {optimal_threshold:.3f}")
print(f"F1-score with optimal threshold: {optimal_f1:.4f}")
print(f"Default threshold F1-score: {f1_opt:.4f}")
print(f"Improvement with threshold optimization: {((optimal_f1 - f1_opt) / f1_opt * 100):.2f}%")

print("\nFinal Classification Report with Optimal Threshold:")
print(classification_report(y_val_opt, y_pred_optimal))
print("\nFinal Confusion Matrix:")
print(confusion_matrix(y_val_opt, y_pred_optimal))


=== THRESHOLD OPTIMIZATION ===
Optimal threshold: 0.512
F1-score with optimal threshold: 0.7475
Default threshold F1-score: 0.7500
Improvement with threshold optimization: -0.34%

Final Classification Report with Optimal Threshold:
              precision    recall  f1-score   support

           0       0.71      0.85      0.77        20
           1       0.81      0.65      0.72        20

    accuracy                           0.75        40
   macro avg       0.76      0.75      0.75        40
weighted avg       0.76      0.75      0.75        40


Final Confusion Matrix:
[[17  3]
 [ 7 13]]


In [39]:
# STRATEGY 8: CROSS-VALIDATION FOR ROBUST EVALUATION

def cross_validate_optimized_model(X, y, cv_folds=5):
    """Perform cross-validation with the optimized pipeline"""
    cv = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
    f1_scores = []
    
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
        print(f"\nFold {fold + 1}/{cv_folds}...")
        
        X_train_cv, X_val_cv = X[train_idx], X[val_idx]
        y_train_cv, y_val_cv = y[train_idx], y[val_idx]
        
        # Apply SMOTE
        smote_cv = SMOTE(random_state=42, k_neighbors=3)
        X_train_balanced, y_train_balanced = smote_cv.fit_resample(X_train_cv, y_train_cv)
        
        # Train ensemble
        ensemble_cv = OptimizedStackedEnsemble()
        X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
            X_train_balanced, y_train_balanced, test_size=0.2, random_state=42, stratify=y_train_balanced
        )
        
        ensemble_cv.fit(X_train_split, y_train_split, X_val_split, y_val_split)
        
        # Predict and calculate F1
        y_proba_cv = ensemble_cv.predict_proba(X_val_cv)[:, 1]
        threshold_cv, _ = optimize_threshold_for_f1(y_val_cv, y_proba_cv)
        y_pred_cv = (y_proba_cv >= threshold_cv).astype(int)
        
        f1_cv = f1_score(y_val_cv, y_pred_cv, average='weighted')
        f1_scores.append(f1_cv)
        print(f"Fold {fold + 1} F1-score: {f1_cv:.4f}")
    
    return f1_scores

# Perform cross-validation
print("\n=== CROSS-VALIDATION EVALUATION ===")
cv_f1_scores = cross_validate_optimized_model(X_engineered, y, cv_folds=5)

print(f"\nCross-validation F1-scores: {[f'{score:.4f}' for score in cv_f1_scores]}")
print(f"Mean F1-score: {np.mean(cv_f1_scores):.4f} ± {np.std(cv_f1_scores):.4f}")
print(f"Best F1-score: {np.max(cv_f1_scores):.4f}")
print(f"Worst F1-score: {np.min(cv_f1_scores):.4f}")


=== CROSS-VALIDATION EVALUATION ===

Fold 1/5...
Training base models...
Training xgb...
Training lgb...
Training rf...
Training svm...
Training mlp...
Training meta-model...
Ensemble training completed!
Fold 1 F1-score: 0.7025

Fold 2/5...
Training base models...
Training xgb...
Training lgb...
Training rf...
Training svm...
Training mlp...
Training meta-model...
Ensemble training completed!
Fold 2 F1-score: 0.7500

Fold 3/5...
Training base models...
Training xgb...
Training lgb...
Training rf...
Training svm...
Training mlp...
Training meta-model...
Ensemble training completed!
Fold 3 F1-score: 0.8249

Fold 4/5...
Training base models...
Training xgb...
Training lgb...
Training rf...
Training svm...
Training mlp...
Training meta-model...
Ensemble training completed!
Fold 4 F1-score: 0.6465

Fold 5/5...
Training base models...
Training xgb...
Training lgb...
Training rf...
Training svm...
Training mlp...
Training meta-model...
Ensemble training completed!
Fold 5 F1-score: 0.6732

Cr

In [40]:
# STRATEGY 9: SAVE OPTIMIZED MODEL AND COMPONENTS

class OptimizedPlagiarismDetector:
    """Complete optimized plagiarism detection pipeline"""
    def __init__(self):
        self.feature_engineer = None
        self.smote = None
        self.ensemble = None
        self.optimal_threshold = 0.5
        
    def fit(self, X_train, y_train):
        """Fit the complete optimized pipeline"""
        # Feature engineering on hold-out train set
        self.feature_engineer = OptimizedFeatureEngineer(n_features=3000)
        X_engineered = self.feature_engineer.fit_transform(X_train, y_train)
        
        # Data balancing
        self.smote = SMOTE(random_state=42, k_neighbors=3)
        X_balanced, y_balanced = self.smote.fit_resample(X_engineered, y_train)
        
        # Internal train/validation split
        X_tr, X_val, y_tr, y_val = train_test_split(
            X_balanced, y_balanced, test_size=0.2, random_state=42, stratify=y_balanced
        )
        
        # Train ensemble
        self.ensemble = OptimizedStackedEnsemble()
        self.ensemble.fit(X_tr, y_tr, X_val, y_val)
        
        # Optimize threshold on validation fold
        y_proba_val = self.ensemble.predict_proba(X_val)[:, 1]
        self.optimal_threshold, _ = optimize_threshold_for_f1(y_val, y_proba_val)
        
        return self
    
    def predict(self, X):
        """Make predictions using the optimized pipeline"""
        X_engineered = self.feature_engineer.transform(X)
        y_proba = self.ensemble.predict_proba(X_engineered)[:, 1]
        return (y_proba >= self.optimal_threshold).astype(int)
    
    def predict_proba(self, X):
        """Predict probabilities using the optimized pipeline"""
        X_engineered = self.feature_engineer.transform(X)
        return self.ensemble.predict_proba(X_engineered)

# Train final optimized model on hold-out train set
print("\n=== TRAINING FINAL OPTIMIZED MODEL (ON HOLD-OUT TRAIN) ===")
final_optimized_model = OptimizedPlagiarismDetector()
final_optimized_model.fit(X_train_holdout, y_train_holdout)

# Evaluate on hold-out test set
print("\n=== EVALUATING ON HOLD-OUT TEST SET ===")
y_pred_holdout = final_optimized_model.predict(X_test_holdout)
final_f1 = f1_score(y_test_holdout, y_pred_holdout, average='weighted')
print(f"\nHOLD-OUT TEST PERFORMANCE:")
print(f"F1-score: {final_f1:.4f}")
print(f"Accuracy: {accuracy_score(y_test_holdout, y_pred_holdout):.4f}")
print("\nClassification Report:")
print(classification_report(y_test_holdout, y_pred_holdout))

# Save the optimized model
os.makedirs('models', exist_ok=True)
with open('models/optimized_plagiarism_detector.pkl', 'wb') as f:
    pickle.dump(final_optimized_model, f)

print("\nOptimized model saved to 'models/optimized_plagiarism_detector.pkl'")
print("\n=== F1-SCORE OPTIMIZATION COMPLETE ===")


=== TRAINING FINAL OPTIMIZED MODEL (ON HOLD-OUT TRAIN) ===
Starting optimized feature engineering...
Features engineered: 10484 -> 10491
Features selected: 10491 -> 3000
Feature scaling completed!
Training base models...
Training xgb...
Training lgb...
Training rf...
Training svm...
Training mlp...
Training meta-model...
Ensemble training completed!

=== EVALUATING ON HOLD-OUT TEST SET ===

HOLD-OUT TEST PERFORMANCE:
F1-score: 0.6992
Accuracy: 0.7000

Classification Report:
              precision    recall  f1-score   support

           0       0.72      0.65      0.68        20
           1       0.68      0.75      0.71        20

    accuracy                           0.70        40
   macro avg       0.70      0.70      0.70        40
weighted avg       0.70      0.70      0.70        40


Optimized model saved to 'models/optimized_plagiarism_detector.pkl'

=== F1-SCORE OPTIMIZATION COMPLETE ===


In [ ]:
# Cell 24: Measure per-model inference time on one sample
import time
import pandas as pd

# Prepare one numeric sample (raw features)
X_sample_raw = X_test_holdout[:1]

# Transform the sample through the feature engineering pipeline
X_sample_engineered = final_optimized_model.feature_engineer.transform(X_sample_raw)

print(f"Raw sample shape: {X_sample_raw.shape}")
print(f"Engineered sample shape: {X_sample_engineered.shape}")

times = []
# Measure inference time for each base model in the ensemble
for name, model in final_optimized_model.ensemble.models.items():
    start = time.time()
    try:
        # All models in the ensemble expect engineered features
        _ = model.predict(X_sample_engineered)
        inference_time = (time.time() - start) * 1000  # Convert to milliseconds
        times.append({'model': name, 'time_ms': inference_time})
        print(f"{name}: {inference_time:.4f}ms")
    except Exception as e:
        print(f"Error with {name}: {e}")
        times.append({'model': name, 'time_ms': None, 'error': str(e)})

# Measure full pipeline inference time
start = time.time()
_ = final_optimized_model.predict(X_sample_raw)
full_pipeline_time = (time.time() - start) * 1000  # Convert to milliseconds
times.append({'model': 'full_pipeline', 'time_ms': full_pipeline_time})

print(f"\nFull pipeline inference time: {full_pipeline_time:.4f}ms")
print("\nTiming Results:")
print(pd.DataFrame(times))

Raw sample shape: (1, 10484)
Engineered sample shape: (1, 3000)
xgb: 0.7918ms
lgb: 1.1246ms
rf: 4.2477ms
svm: 0.8550ms
mlp: 0.4508ms

Full pipeline inference time: 100.0836ms

Timing Results:
           model     time_ms
0            xgb    0.791788
1            lgb    1.124620
2             rf    4.247665
3            svm    0.854969
4            mlp    0.450850
5  full_pipeline  100.083590
